## Pretrained VGGNet

In [1]:
# Operations for floydhub
# !git clone https://github.com/machrisaa/tensorflow-vgg tensorflow_vgg
# !ln -s /data .

Cloning into 'tensorflow_vgg'...
remote: Counting objects: 113, done.
remote: Total 113 (delta 0), reused 0 (delta 0), pack-reused 113
Receiving objects: 100% (113/113), 55.91 KiB | 0 bytes/s, done.
Resolving deltas: 100% (61/61), done.
Checking connectivity... done.


In [1]:
from urllib.request import urlretrieve
from os.path import isfile, isdir
from tqdm import tqdm

vgg_dir = 'tensorflow_vgg/'
vgg_name = 'vgg19'

# Make sure vgg exists
if not isdir(vgg_dir):
    raise Exception("VGG directory doesn't exist!")

class DLProgress(tqdm):
    last_block = 0

    def hook(self, block_num=1, block_size=1, total_size=None):
        self.total = total_size
        self.update((block_num - self.last_block) * block_size)
        self.last_block = block_num

vgg_param_file = '{}{}.npy'.format(vgg_dir, vgg_name)
if not isfile(vgg_param_file):
    with DLProgress(unit='B', unit_scale=True, miniters=1, desc= vgg_name + ' Parameters') as pbar:
        urlretrieve(
            'https://s3.amazonaws.com/content.udacity-data.com/nd101/{}.npy'.format(vgg_name),
            vgg_param_file,
            pbar.hook)
else:
    print("Parameter file already exists!")

Parameter file already exists!


In [2]:
import os

import numpy as np
import tensorflow as tf

from tensorflow_vgg import vgg19
# from tensorflow_vgg import vgg16
from tensorflow_vgg import utils

In [3]:
data_dir = 'data/'
train_dir = data_dir + 'train/'

classes = [d for d in os.listdir(train_dir) if os.path.isdir(train_dir + d)]

In [4]:
from scipy.ndimage.interpolation import rotate
from skimage.transform import resize

def horizontal_flip(image):
    image = image[:, ::-1, :]
    return image

def vertical_flip(image):
    image = image[::-1, :, :]
    return image

def random_rotation(image, angle_range=(0, 180)):
    h, w, _ = image.shape
    angle = np.random.randint(*angle_range)
    image = rotate(image, angle)
    image = resize(image, (h, w))
    return image

In [5]:
import csv

batch_size = 20
batch = []

with tf.Session() as sess:
    vgg = vgg19.Vgg19()
    # vgg = vgg16.Vgg16()
    input_ = tf.placeholder(tf.float32, [None, 224, 224, 3])
    
    with tf.name_scope("content_vgg"):
        vgg.build(input_)
    
    
    codes = None
    labels = []
    count = 0

    for d_type in ['train', 'valid']:
        for c in classes:
            image_dir = '{}{}/{}/'.format(data_dir, d_type, c) # e.g. data/train/melanoma/
            files = os.listdir(image_dir)
            for i, file in enumerate(files, 1):
                # load image and resize it to 224x224
                file_path = os.path.join(image_dir, file)
                img = utils.load_image(file_path)
                batch.append(img.reshape((1, 224, 224, 3)))
                labels.append(c)
                
                # data augumentation for all training data but nevus
                if d_type == 'train':
                    batch.append(horizontal_flip(img).reshape((1,224,224,3)))
                    labels.append(c)
                    batch.append(vertical_flip(img).reshape((1,224,224,3)))
                    labels.append(c)
                    batch.append(random_rotation(img).reshape((1,224,224,3)))
                    labels.append(c)
                    batch.append(random_rotation(img, angle_range=(-180, 0)).reshape((1,224,224,3)))
                    labels.append(c)                  

                if (len(batch) >= batch_size) or i == len(files):
                    images = np.concatenate(batch)

                    feed_dict = {input_: images}
                    codes_batch = sess.run(vgg.relu6, feed_dict=feed_dict)

                    if codes is None:
                        codes = codes_batch
                    else:
                        codes = np.concatenate((codes, codes_batch))

                    count += len(batch)
                    batch = []
                    print('data: {}, class: {}, {} / {} images processed. data count: {}'.format(d_type, c, i, len(files), count))

        # write codes to file
        with open('{}_{}_codes'.format(vgg_name, d_type), 'w') as f:
            codes.tofile(f)
            codes = None

        # write labels to file
        with open('{}_{}_labels'.format(vgg_name, d_type), 'w') as f:
            writer = csv.writer(f, delimiter='\n')
            writer.writerow(labels)
            labels = []

        count = 0

/Users/junji/Development/udacity-deeplearning/dermatologist-ai/tensorflow_vgg/vgg19.npy
npy file loaded
build model started
build model finished: 0s


/Users/junji/miniconda3/envs/dermatologist-ai/lib/python3.5/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


data: train, class: melanoma, 4 / 374 images processed. data count: 20
data: train, class: melanoma, 8 / 374 images processed. data count: 40
data: train, class: melanoma, 12 / 374 images processed. data count: 60
data: train, class: melanoma, 16 / 374 images processed. data count: 80
data: train, class: melanoma, 20 / 374 images processed. data count: 100
data: train, class: melanoma, 24 / 374 images processed. data count: 120
data: train, class: melanoma, 28 / 374 images processed. data count: 140
data: train, class: melanoma, 32 / 374 images processed. data count: 160
data: train, class: melanoma, 36 / 374 images processed. data count: 180
data: train, class: melanoma, 40 / 374 images processed. data count: 200
data: train, class: melanoma, 44 / 374 images processed. data count: 220
data: train, class: melanoma, 48 / 374 images processed. data count: 240
data: train, class: melanoma, 52 / 374 images processed. data count: 260
data: train, class: melanoma, 56 / 374 images processed. 

data: train, class: nevus, 360 / 1372 images processed. data count: 2230
data: train, class: nevus, 380 / 1372 images processed. data count: 2250
data: train, class: nevus, 400 / 1372 images processed. data count: 2270
data: train, class: nevus, 420 / 1372 images processed. data count: 2290
data: train, class: nevus, 440 / 1372 images processed. data count: 2310
data: train, class: nevus, 460 / 1372 images processed. data count: 2330
data: train, class: nevus, 480 / 1372 images processed. data count: 2350
data: train, class: nevus, 500 / 1372 images processed. data count: 2370
data: train, class: nevus, 520 / 1372 images processed. data count: 2390
data: train, class: nevus, 540 / 1372 images processed. data count: 2410
data: train, class: nevus, 560 / 1372 images processed. data count: 2430
data: train, class: nevus, 580 / 1372 images processed. data count: 2450
data: train, class: nevus, 600 / 1372 images processed. data count: 2470
data: train, class: nevus, 620 / 1372 images proces

data: train, class: seborrheic_keratosis, 208 / 254 images processed. data count: 4282
data: train, class: seborrheic_keratosis, 212 / 254 images processed. data count: 4302
data: train, class: seborrheic_keratosis, 216 / 254 images processed. data count: 4322
data: train, class: seborrheic_keratosis, 220 / 254 images processed. data count: 4342
data: train, class: seborrheic_keratosis, 224 / 254 images processed. data count: 4362
data: train, class: seborrheic_keratosis, 228 / 254 images processed. data count: 4382
data: train, class: seborrheic_keratosis, 232 / 254 images processed. data count: 4402
data: train, class: seborrheic_keratosis, 236 / 254 images processed. data count: 4422
data: train, class: seborrheic_keratosis, 240 / 254 images processed. data count: 4442
data: train, class: seborrheic_keratosis, 244 / 254 images processed. data count: 4462
data: train, class: seborrheic_keratosis, 248 / 254 images processed. data count: 4482
data: train, class: seborrheic_keratosis, 2

## Building the Classifier

In [5]:
# read codes and labels from file
import csv

# train data
with open('{}_train_labels'.format(vgg_name)) as f:
    reader = csv.reader(f, delimiter='\n')
    train_labels = np.array([each for each in reader if len(each) > 0]).squeeze()
with open('{}_train_codes'.format(vgg_name)) as f:
    train_x = np.fromfile(f, dtype=np.float32)
    train_x = train_x.reshape((len(train_labels), -1))
    
# valid data
with open('{}_valid_labels'.format(vgg_name)) as f:
    reader = csv.reader(f, delimiter='\n')
    valid_labels = np.array([each for each in reader if len(each) > 0]).squeeze()
with open('{}_valid_codes'.format(vgg_name)) as f:
    val_x = np.fromfile(f, dtype=np.float32)
    val_x = val_x.reshape((len(valid_labels), -1))

In [6]:
from sklearn.preprocessing import LabelBinarizer

lb = LabelBinarizer()
lb.fit(classes)

LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False)

In [7]:
train_y = lb.transform(train_labels)
val_y = lb.transform(valid_labels)

In [8]:
print("Train shapes (x, y):", train_x.shape, train_y.shape)
print("Validation shapes (x, y):", val_x.shape, val_y.shape)

Train shapes (x, y): (10000, 4096) (10000, 3)
Validation shapes (x, y): (150, 4096) (150, 3)


In [9]:
inputs_ = tf.placeholder(tf.float32, shape=[None, train_x.shape[1]])
labels_ = tf.placeholder(tf.int64, shape=[None, train_y.shape[1]])

fc_1 = tf.contrib.layers.fully_connected(inputs_, 512)
dropout_1 = tf.contrib.layers.dropout(fc_1)
fc_2 = tf.contrib.layers.fully_connected(dropout_1, 128)
dropout_2 = tf.contrib.layers.dropout(fc_2)
    
logits = tf.contrib.layers.fully_connected(dropout_2, train_y.shape[1], activation_fn=None)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=labels_, logits=logits)
cost = tf.reduce_mean(cross_entropy)

optimizer = tf.train.RMSPropOptimizer(0.001).minimize(cost)

predicted = tf.nn.softmax(logits)
correct_pred = tf.equal(tf.argmax(predicted, 1), tf.argmax(labels_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [10]:
def get_batches(x, y, n_batches=10):
    """ Return a generator that yields batches from arrays x and y. """
    batch_size = len(x)//n_batches
    
    for ii in range(0, n_batches*batch_size, batch_size):
        # If we're not on the last batch, grab data with size batch_size
        if ii != (n_batches-1)*batch_size:
            X, Y = x[ii: ii+batch_size], y[ii: ii+batch_size] 
        # On the last batch, grab the rest of the data
        else:
            X, Y = x[ii:], y[ii:]
        # I love generators
        yield X, Y

In [15]:
import sklearn
train_x_shuffled, train_y_shuffled = sklearn.utils.shuffle(train_x, train_y)

## Training

In [16]:
!mkdir checkpoints

epochs = 200
iteration = 0
saver = tf.train.Saver()
with tf.Session() as sess:
    
    sess.run(tf.global_variables_initializer())
    # saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    for e in range(epochs):
        for x, y in get_batches(train_x_shuffled, train_y_shuffled, n_batches=5):
            feed = {inputs_: x,
                    labels_: y}
            loss, _ = sess.run([cost, optimizer], feed_dict=feed)
            print("Epoch: {}/{}".format(e+1, epochs),
                  "Iteration: {}".format(iteration),
                  "Training loss: {:.5f}".format(loss))
            iteration += 1
            
            if iteration % 5 == 0:
                feed = {inputs_: val_x, labels_: val_y}
                val_acc = sess.run(accuracy, feed_dict=feed)
                print("Epoch: {}/{}".format(e, epochs),
                      "Iteration: {}".format(iteration),
                      "Validation Acc: {:.4f}".format(val_acc))
    saver.save(sess, "checkpoints/skin_diseases.ckpt")

mkdir: checkpoints: File exists
Epoch: 1/200 Iteration: 0 Training loss: 4.79246
Epoch: 1/200 Iteration: 1 Training loss: 3.98259
Epoch: 1/200 Iteration: 2 Training loss: 3.71347
Epoch: 1/200 Iteration: 3 Training loss: 3.64130
Epoch: 1/200 Iteration: 4 Training loss: 3.55263
Epoch: 0/200 Iteration: 5 Validation Acc: 0.4800
Epoch: 2/200 Iteration: 5 Training loss: 3.55986
Epoch: 2/200 Iteration: 6 Training loss: 3.37187
Epoch: 2/200 Iteration: 7 Training loss: 3.22494
Epoch: 2/200 Iteration: 8 Training loss: 3.24324
Epoch: 2/200 Iteration: 9 Training loss: 2.89744
Epoch: 1/200 Iteration: 10 Validation Acc: 0.4000
Epoch: 3/200 Iteration: 10 Training loss: 3.07067
Epoch: 3/200 Iteration: 11 Training loss: 2.91820
Epoch: 3/200 Iteration: 12 Training loss: 2.62414
Epoch: 3/200 Iteration: 13 Training loss: 2.69203
Epoch: 3/200 Iteration: 14 Training loss: 2.55110
Epoch: 2/200 Iteration: 15 Validation Acc: 0.3800
Epoch: 4/200 Iteration: 15 Training loss: 2.48756
Epoch: 4/200 Iteration: 16 Tr

Epoch: 28/200 Iteration: 135 Training loss: 0.76384
Epoch: 28/200 Iteration: 136 Training loss: 0.81519
Epoch: 28/200 Iteration: 137 Training loss: 0.83464
Epoch: 28/200 Iteration: 138 Training loss: 1.17708
Epoch: 28/200 Iteration: 139 Training loss: 1.10830
Epoch: 27/200 Iteration: 140 Validation Acc: 0.4667
Epoch: 29/200 Iteration: 140 Training loss: 1.62440
Epoch: 29/200 Iteration: 141 Training loss: 1.01107
Epoch: 29/200 Iteration: 142 Training loss: 0.84105
Epoch: 29/200 Iteration: 143 Training loss: 0.80321
Epoch: 29/200 Iteration: 144 Training loss: 0.73977
Epoch: 28/200 Iteration: 145 Validation Acc: 0.5467
Epoch: 30/200 Iteration: 145 Training loss: 0.76020
Epoch: 30/200 Iteration: 146 Training loss: 0.72818
Epoch: 30/200 Iteration: 147 Training loss: 0.75727
Epoch: 30/200 Iteration: 148 Training loss: 0.79500
Epoch: 30/200 Iteration: 149 Training loss: 0.77598
Epoch: 29/200 Iteration: 150 Validation Acc: 0.6267
Epoch: 31/200 Iteration: 150 Training loss: 0.76522
Epoch: 31/20

Epoch: 54/200 Iteration: 267 Training loss: 0.55088
Epoch: 54/200 Iteration: 268 Training loss: 0.55518
Epoch: 54/200 Iteration: 269 Training loss: 0.54586
Epoch: 53/200 Iteration: 270 Validation Acc: 0.6800
Epoch: 55/200 Iteration: 270 Training loss: 0.59734
Epoch: 55/200 Iteration: 271 Training loss: 0.68161
Epoch: 55/200 Iteration: 272 Training loss: 0.69412
Epoch: 55/200 Iteration: 273 Training loss: 0.77622
Epoch: 55/200 Iteration: 274 Training loss: 0.61714
Epoch: 54/200 Iteration: 275 Validation Acc: 0.6200
Epoch: 56/200 Iteration: 275 Training loss: 0.54596
Epoch: 56/200 Iteration: 276 Training loss: 0.53029
Epoch: 56/200 Iteration: 277 Training loss: 0.50688
Epoch: 56/200 Iteration: 278 Training loss: 0.53110
Epoch: 56/200 Iteration: 279 Training loss: 0.55957
Epoch: 55/200 Iteration: 280 Validation Acc: 0.6467
Epoch: 57/200 Iteration: 280 Training loss: 0.71577
Epoch: 57/200 Iteration: 281 Training loss: 0.63653
Epoch: 57/200 Iteration: 282 Training loss: 0.57093
Epoch: 57/20

Epoch: 80/200 Iteration: 399 Training loss: 0.42556
Epoch: 79/200 Iteration: 400 Validation Acc: 0.6400
Epoch: 81/200 Iteration: 400 Training loss: 0.40652
Epoch: 81/200 Iteration: 401 Training loss: 0.37123
Epoch: 81/200 Iteration: 402 Training loss: 0.35470
Epoch: 81/200 Iteration: 403 Training loss: 0.40832
Epoch: 81/200 Iteration: 404 Training loss: 0.43886
Epoch: 80/200 Iteration: 405 Validation Acc: 0.6267
Epoch: 82/200 Iteration: 405 Training loss: 0.68433
Epoch: 82/200 Iteration: 406 Training loss: 0.59047
Epoch: 82/200 Iteration: 407 Training loss: 0.44953
Epoch: 82/200 Iteration: 408 Training loss: 0.41733
Epoch: 82/200 Iteration: 409 Training loss: 0.38410
Epoch: 81/200 Iteration: 410 Validation Acc: 0.6733
Epoch: 83/200 Iteration: 410 Training loss: 0.39145
Epoch: 83/200 Iteration: 411 Training loss: 0.42987
Epoch: 83/200 Iteration: 412 Training loss: 0.57967
Epoch: 83/200 Iteration: 413 Training loss: 0.65390
Epoch: 83/200 Iteration: 414 Training loss: 0.42957
Epoch: 82/20

Epoch: 107/200 Iteration: 530 Training loss: 0.41336
Epoch: 107/200 Iteration: 531 Training loss: 0.32847
Epoch: 107/200 Iteration: 532 Training loss: 0.27291
Epoch: 107/200 Iteration: 533 Training loss: 0.27595
Epoch: 107/200 Iteration: 534 Training loss: 0.24448
Epoch: 106/200 Iteration: 535 Validation Acc: 0.6267
Epoch: 108/200 Iteration: 535 Training loss: 0.26281
Epoch: 108/200 Iteration: 536 Training loss: 0.23485
Epoch: 108/200 Iteration: 537 Training loss: 0.26329
Epoch: 108/200 Iteration: 538 Training loss: 0.33718
Epoch: 108/200 Iteration: 539 Training loss: 0.39721
Epoch: 107/200 Iteration: 540 Validation Acc: 0.5867
Epoch: 109/200 Iteration: 540 Training loss: 0.43488
Epoch: 109/200 Iteration: 541 Training loss: 0.37080
Epoch: 109/200 Iteration: 542 Training loss: 0.31992
Epoch: 109/200 Iteration: 543 Training loss: 0.35536
Epoch: 109/200 Iteration: 544 Training loss: 0.31673
Epoch: 108/200 Iteration: 545 Validation Acc: 0.6133
Epoch: 110/200 Iteration: 545 Training loss: 0

Epoch: 133/200 Iteration: 660 Training loss: 0.23335
Epoch: 133/200 Iteration: 661 Training loss: 0.20268
Epoch: 133/200 Iteration: 662 Training loss: 0.21280
Epoch: 133/200 Iteration: 663 Training loss: 0.26080
Epoch: 133/200 Iteration: 664 Training loss: 0.23568
Epoch: 132/200 Iteration: 665 Validation Acc: 0.6200
Epoch: 134/200 Iteration: 665 Training loss: 0.25542
Epoch: 134/200 Iteration: 666 Training loss: 0.21819
Epoch: 134/200 Iteration: 667 Training loss: 0.20817
Epoch: 134/200 Iteration: 668 Training loss: 0.20798
Epoch: 134/200 Iteration: 669 Training loss: 0.18078
Epoch: 133/200 Iteration: 670 Validation Acc: 0.6800
Epoch: 135/200 Iteration: 670 Training loss: 0.22496
Epoch: 135/200 Iteration: 671 Training loss: 0.19126
Epoch: 135/200 Iteration: 672 Training loss: 0.20262
Epoch: 135/200 Iteration: 673 Training loss: 0.19960
Epoch: 135/200 Iteration: 674 Training loss: 0.21265
Epoch: 134/200 Iteration: 675 Validation Acc: 0.6400
Epoch: 136/200 Iteration: 675 Training loss: 0

Epoch: 159/200 Iteration: 790 Training loss: 0.13555
Epoch: 159/200 Iteration: 791 Training loss: 0.13805
Epoch: 159/200 Iteration: 792 Training loss: 0.15332
Epoch: 159/200 Iteration: 793 Training loss: 0.18491
Epoch: 159/200 Iteration: 794 Training loss: 0.21175
Epoch: 158/200 Iteration: 795 Validation Acc: 0.6533
Epoch: 160/200 Iteration: 795 Training loss: 0.26171
Epoch: 160/200 Iteration: 796 Training loss: 0.29752
Epoch: 160/200 Iteration: 797 Training loss: 0.36227
Epoch: 160/200 Iteration: 798 Training loss: 0.79201
Epoch: 160/200 Iteration: 799 Training loss: 0.45897
Epoch: 159/200 Iteration: 800 Validation Acc: 0.6133
Epoch: 161/200 Iteration: 800 Training loss: 0.36252
Epoch: 161/200 Iteration: 801 Training loss: 0.23458
Epoch: 161/200 Iteration: 802 Training loss: 0.18659
Epoch: 161/200 Iteration: 803 Training loss: 0.18833
Epoch: 161/200 Iteration: 804 Training loss: 0.16194
Epoch: 160/200 Iteration: 805 Validation Acc: 0.6600
Epoch: 162/200 Iteration: 805 Training loss: 0

Epoch: 185/200 Iteration: 920 Training loss: 0.18087
Epoch: 185/200 Iteration: 921 Training loss: 0.19412
Epoch: 185/200 Iteration: 922 Training loss: 0.18582
Epoch: 185/200 Iteration: 923 Training loss: 0.24822
Epoch: 185/200 Iteration: 924 Training loss: 0.21198
Epoch: 184/200 Iteration: 925 Validation Acc: 0.6400
Epoch: 186/200 Iteration: 925 Training loss: 0.19233
Epoch: 186/200 Iteration: 926 Training loss: 0.13224
Epoch: 186/200 Iteration: 927 Training loss: 0.12952
Epoch: 186/200 Iteration: 928 Training loss: 0.13270
Epoch: 186/200 Iteration: 929 Training loss: 0.11399
Epoch: 185/200 Iteration: 930 Validation Acc: 0.6533
Epoch: 187/200 Iteration: 930 Training loss: 0.13242
Epoch: 187/200 Iteration: 931 Training loss: 0.13721
Epoch: 187/200 Iteration: 932 Training loss: 0.17943
Epoch: 187/200 Iteration: 933 Training loss: 0.21733
Epoch: 187/200 Iteration: 934 Training loss: 0.18053
Epoch: 186/200 Iteration: 935 Validation Acc: 0.6067
Epoch: 188/200 Iteration: 935 Training loss: 0

## Export result to csv

In [17]:
import csv

predections_csv = open('predections.csv', 'w')
writer = csv.writer(predections_csv, lineterminator='\n')
writer.writerow(['Id', 'task_1', 'task_2'])

batch_size = 20
batch = []
labels_batch = []
files_batch = []

d_type = 'test'
    
saver = tf.train.Saver()
with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    vgg = vgg19.Vgg19()
    # vgg = vgg16.Vgg16()
    input_ = tf.placeholder(tf.float32, [None, 224, 224, 3])
    
    with tf.name_scope("content_vgg"):
        vgg.build(input_)
    
    for c in classes:
        image_dir = '{}{}/{}/'.format(data_dir, d_type, c) # e.g. data/train/melanoma/
        files = os.listdir(image_dir)
        for i, file in enumerate(files, 1):
            # load image and resize it to 224x224
            file_path = os.path.join(image_dir, file)
            img = utils.load_image(file_path)
            
            batch.append(img.reshape((1, 224, 224, 3)))
            labels_batch.append(c)
            files_batch.append(file_path)

            if (len(batch) >= batch_size) or i == len(files):
                images = np.concatenate(batch)

                feed_dict = {input_: images}
                codes_batch = sess.run(vgg.relu6, feed_dict=feed_dict)
                
                feed_dict = {inputs_: codes_batch, labels_: lb.transform(labels_batch)}
                predections_batch = sess.run(predicted, feed_dict=feed_dict)

                for ii in range(len(batch)):
                    predection = predections_batch[ii]
                    label = labels_batch[ii]
                    file_id = files_batch[ii]
                    
                    p_melanoma = predection[0]
                    p_seborrheic_keratosis = predection[2]
                    
                    print(file_id, p_melanoma, p_seborrheic_keratosis)
                    writer.writerow([file_id, p_melanoma, p_seborrheic_keratosis])
                    
                batch = []
                labels_batch = []
                files_batch = []
                print('data: {}, class: {}, {} / {} images processed.'.format(d_type, c, i, len(files)))
                
                
predections_csv.close()

INFO:tensorflow:Restoring parameters from checkpoints/skin_diseases.ckpt
/Users/junji/Development/udacity-deeplearning/dermatologist-ai/tensorflow_vgg/vgg19.npy
npy file loaded
build model started
build model finished: 1s


/Users/junji/miniconda3/envs/dermatologist-ai/lib/python3.5/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


data/test/melanoma/ISIC_0012258.jpg 0.00204268 0.0625146
data/test/melanoma/ISIC_0012356.jpg 0.00703594 0.0762861
data/test/melanoma/ISIC_0012369.jpg 6.0923e-06 3.03647e-07
data/test/melanoma/ISIC_0012395.jpg 6.60503e-10 1.0
data/test/melanoma/ISIC_0012425.jpg 6.03124e-14 1.86998e-11
data/test/melanoma/ISIC_0012758.jpg 0.000252329 2.64962e-10
data/test/melanoma/ISIC_0012989.jpg 0.999877 5.14157e-17
data/test/melanoma/ISIC_0013072.jpg 0.586808 0.0919377
data/test/melanoma/ISIC_0013073.jpg 0.899309 0.0992762
data/test/melanoma/ISIC_0013242.jpg 0.00076747 0.0850101
data/test/melanoma/ISIC_0013277.jpg 0.533598 0.306941
data/test/melanoma/ISIC_0013321.jpg 1.60834e-07 1.0
data/test/melanoma/ISIC_0013374.jpg 0.998755 0.00102823
data/test/melanoma/ISIC_0013411.jpg 7.12369e-07 0.877945
data/test/melanoma/ISIC_0013414.jpg 3.69211e-10 0.0133549
data/test/melanoma/ISIC_0013455.jpg 0.276062 7.09349e-05
data/test/melanoma/ISIC_0013457.jpg 0.999285 0.000101422
data/test/melanoma/ISIC_0013459.jpg 0.85

data/test/nevus/ISIC_0013602.jpg 0.89414 2.6363e-12
data/test/nevus/ISIC_0013738.jpg 1.31819e-17 3.1691e-15
data/test/nevus/ISIC_0013794.jpg 1.97102e-07 1.34225e-05
data/test/nevus/ISIC_0013809.jpg 0.898028 0.00108071
data/test/nevus/ISIC_0013891.jpg 5.95804e-07 4.84833e-07
data/test/nevus/ISIC_0013897.jpg 0.998271 0.00157174
data/test/nevus/ISIC_0013911.jpg 1.53179e-10 1.0
data/test/nevus/ISIC_0013966.jpg 0.000429395 0.0163258
data/test/nevus/ISIC_0013998.jpg 2.46588e-05 4.06155e-05
data/test/nevus/ISIC_0014090.jpg 2.98216e-08 1.9876e-06
data/test/nevus/ISIC_0014117.jpg 0.00077733 1.19909e-05
data/test/nevus/ISIC_0014470.jpg 1.79728e-07 0.999287
data/test/nevus/ISIC_0014675.jpg 2.57421e-12 2.56436e-23
data/test/nevus/ISIC_0014677.jpg 3.27962e-18 3.61391e-22
data/test/nevus/ISIC_0014687.jpg 2.45273e-07 3.59259e-06
data/test/nevus/ISIC_0014693.jpg 8.63434e-05 0.999838
data/test/nevus/ISIC_0014697.jpg 7.97103e-21 8.17906e-32
data/test/nevus/ISIC_0014698.jpg 1.11887e-16 4.29445e-20
data/t

data/test/nevus/ISIC_0015274.jpg 0.548173 0.00251342
data/test/nevus/ISIC_0015276.jpg 0.00200946 1.13549e-11
data/test/nevus/ISIC_0015279.jpg 1.63765e-30 2.05913e-19
data/test/nevus/ISIC_0015283.jpg 1.24228e-22 3.26291e-24
data/test/nevus/ISIC_0015291.jpg 3.06128e-10 5.11164e-12
data/test/nevus/ISIC_0015293.jpg 1.43712e-07 0.0603334
data/test/nevus/ISIC_0015298.jpg 6.06262e-10 3.49496e-14
data/test/nevus/ISIC_0015309.jpg 1.16359e-12 7.00978e-09
data/test/nevus/ISIC_0015310.jpg 7.11825e-14 3.41911e-12
data/test/nevus/ISIC_0015311.jpg 0.196296 0.463231
data/test/nevus/ISIC_0015312.jpg 1.60593e-05 0.000206785
data/test/nevus/ISIC_0015330.jpg 4.48894e-14 6.23708e-25
data/test/nevus/ISIC_0015331.jpg 2.41576e-14 2.31003e-13
data/test/nevus/ISIC_0015347.jpg 2.30685e-05 1.37014e-08
data/test/nevus/ISIC_0015353.jpg 4.85778e-06 0.974104
data/test/nevus/ISIC_0015355.jpg 7.99663e-18 2.40351e-10
data/test/nevus/ISIC_0015357.jpg 8.22474e-16 2.77651e-20
data/test/nevus/ISIC_0015360.jpg 0.00180071 2.9

data/test/nevus/ISIC_0016037.jpg 0.00125209 0.000755699
data/test/nevus/ISIC_0016038.jpg 5.34933e-11 3.81301e-09
data/test/nevus/ISIC_0016040.jpg 0.000252336 0.00344919
data/test/nevus/ISIC_0016041.jpg 1.0 8.24922e-15
data/test/nevus/ISIC_0016042.jpg 4.01912e-06 0.000268892
data/test/nevus/ISIC_0016043.jpg 9.82865e-12 1.48007e-06
data/test/nevus/ISIC_0016044.jpg 3.45603e-18 0.000117393
data/test/nevus/ISIC_0016045.jpg 0.743939 0.00722481
data/test/nevus/ISIC_0016046.jpg 0.236138 0.000124488
data/test/nevus/ISIC_0016048.jpg 0.00337243 5.86765e-06
data/test/nevus/ISIC_0016049.jpg 0.255361 1.32919e-09
data/test/nevus/ISIC_0016050.jpg 0.00392811 0.00283341
data/test/nevus/ISIC_0016051.jpg 4.94773e-06 0.00140405
data/test/nevus/ISIC_0016052.jpg 0.0172655 0.000213718
data/test/nevus/ISIC_0016053.jpg 7.38449e-07 1.91405e-09
data/test/nevus/ISIC_0016054.jpg 0.00122573 0.99857
data/test/nevus/ISIC_0016055.jpg 7.57024e-06 1.25871e-11
data/test/nevus/ISIC_0016056.jpg 0.0951181 0.058071
data/test/